In [1]:
import pandas as pd
import numpy as np
import random
import ast

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

# input and output

In [2]:
###input
cut_off = 0.3
num = 50
num_process = 50
cut_off_path = f'../../../Results/not_lipid/top{num}_{cut_off}_re/'
yeast870_path =  '../../../Data/model/yeast-GEM.yml'
miss_met_id_file = '../../../Data/model/miss_met_id.json'
metabolites_info_to_GEM_path = cut_off_path + f'metabolites_info_to_GEM_top{num}_{cut_off}.csv'
rxndb_total_info_to_model_espfiltergene_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_espfiltergene.csv'
target_model_smiles_path = '../../../Results/not_lipid/yeast_met_not_lipid.pickle'
not_lipid_target_smiles_path = '../../../Results/not_lipid/target_smiles_not_lipid.pickle'
# rxndb_total_info_to_model_espfiltergene_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_espfiltergene.csv'
proportion = 0.2  # 抽取比例，80%

## output
rxndb_total_info_to_model_espfiltergene_randomfiltermet_path = cut_off_path + f'rxndb_total_info_to_model_top{num}_{cut_off}_espfiltergene_randomfiltermet_{proportion}.csv'

In [3]:
metabolites_info_to_GEM =  pd.read_csv(metabolites_info_to_GEM_path)
metabolites_info_to_GEM.head(3)

,new_met_smiles,sim_smile,ID,compartment
0,CCCCCCCCCCCCCC(=O)[O-],CCCCCCCCCCCCCC(=O)O,s_1161,c
1,NC(Cc1cnc[nH]1)C(=O)O,NC(Cc1cnc[nH]1)C(=O)O,s_1006,c
2,CSCCC(=O)C(=O)COP(=O)(O)O,CSCCC(=O)C(=O)COP(=O)(O)O,s_0311,c


In [4]:
new_met_smiles_dict = {}
for index,row in metabolites_info_to_GEM[metabolites_info_to_GEM['ID'].str.startswith('sn')].iterrows():
    new_met_smiles_dict[row['ID']] = row['new_met_smiles']

In [5]:
# 读取 .pickle 文件
with open(not_lipid_target_smiles_path, 'rb') as file:
    not_lipid_target_smiles = pickle.load(file)

In [6]:
IDlist = []
for id,smiles in tqdm(new_met_smiles_dict.items(),total=len(new_met_smiles_dict)):
    if any(compare_smiles_inchikey(smiles,x) for x in not_lipid_target_smiles):
        IDlist.append(id)

100%|██████████| 418/418 [02:41<00:00,  2.59it/s]


In [7]:
sample_size = int(len(IDlist) * proportion)
drop_IDlist = random.sample(IDlist, sample_size)
print(drop_IDlist)

['sn_40', 'sn_304', 'sn_69', 'sn_29', 'sn_328', 'sn_291', 'sn_349', 'sn_400', 'sn_221', 'sn_321', 'sn_160', 'sn_41', 'sn_303', 'sn_312', 'sn_325', 'sn_187', 'sn_319', 'sn_184', 'sn_235', 'sn_14', 'sn_359', 'sn_377', 'sn_196', 'sn_84', 'sn_294', 'sn_51', 'sn_292', 'sn_383', 'sn_18', 'sn_310', 'sn_109', 'sn_149', 'sn_265', 'sn_318', 'sn_391', 'sn_399', 'sn_311', 'sn_8', 'sn_277', 'sn_282']


In [8]:
rxndb_total_info_to_model_espfiltergene = pd.read_csv(rxndb_total_info_to_model_espfiltergene_path)
rxndb_total_info_to_model_espfiltergene.head(3)

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn19747,1.14.13,MNXR180996_reverse,COc1cc(CC(=O)[O-])ccc1O.O>>COc1cc(C(C)=O)ccc1O...,COc1cc(CC(=O)[O-])ccc1O.O.NC(=O)c1ccc[n+]([C@@...,(YOL114C) or (YCL059C) or (YDR402C) or (YDR534...,"{'sn_256': -1, 's_0803': -1, 's_1198': -1, 'sn...",sn_256 + s_0803 + s_1198 => sn_226 + s_12...
1,rxn19748,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_2...
2,rxn19749,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...


In [9]:
rxndb_total_info_to_model_espfiltergene['equation_dict'] = rxndb_total_info_to_model_espfiltergene['equation_dict'].apply(ast.literal_eval)
rxndb_total_info_to_model_espfiltergene.head(3)

,NO,EC_number,templateID,rxn_smiles_basic,rxn_smiles_final,GPR,equation_dict,equation
0,rxn19747,1.14.13,MNXR180996_reverse,COc1cc(CC(=O)[O-])ccc1O.O>>COc1cc(C(C)=O)ccc1O...,COc1cc(CC(=O)[O-])ccc1O.O.NC(=O)c1ccc[n+]([C@@...,(YOL114C) or (YCL059C) or (YDR402C) or (YDR534...,"{'sn_256': -1, 's_0803': -1, 's_1198': -1, 'sn...",sn_256 + s_0803 + s_1198 => sn_226 + s_12...
1,rxn19748,1.14.13,MNXR149799,CCCCCCCCCCCC(=O)[O-].O=O>>CCCCCCCCCC(O)CC(=O)[...,CCCCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_1065': -1, 's_1275': -1, 's_0794': -1, 's_...",s_1065 + s_1275 + s_0794 + s_1212 => sn_2...
2,rxn19749,1.14.13,MNXR149799,CCCCCCCCCC(=O)[O-].O=O>>CCCCCCCC(O)CC(=O)[O-].O,CCCCCCCCCC(=O)[O-].O=O.[H+].NC(=O)C1=CN([C@@H]...,(YDR402C) or (YDR534C) or (YAL064W) or (YJL151...,"{'s_0595': -1, 's_1275': -1, 's_0794': -1, 's_...",s_0595 + s_1275 + s_0794 + s_1212 => sn_1...


In [10]:
print(rxndb_total_info_to_model_espfiltergene.shape)
rxndb_total_info_to_model_espfiltergene = rxndb_total_info_to_model_espfiltergene[rxndb_total_info_to_model_espfiltergene['equation_dict'].apply(lambda x: all(id not in drop_IDlist for id,_ in x.items()))]
print(rxndb_total_info_to_model_espfiltergene.shape)

(7889, 8)
(7440, 8)


In [11]:
rxndb_total_info_to_model_espfiltergene.reset_index(drop=True,inplace=True)

In [12]:
rxndb_total_info_to_model_espfiltergene.to_csv(rxndb_total_info_to_model_espfiltergene_randomfiltermet_path,index=None)